In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from fairdata import FairData

In [2]:
df_raw = pd.read_csv('data/COMPAS/compas-scores-two-years.csv')
df_raw = df_raw.loc[:,[
    'sex', 'race', 'age', 'juv_fel_count','juv_misd_count',
    'juv_other_count', 'priors_count', 'two_year_recid'
]]
df_raw = df_raw.loc[df_raw['race'].isin(['African-American', 'Caucasian', 'Hispanic']), :]

In [3]:
df_raw

,sex,race,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,two_year_recid
1,Male,African-American,34,0,0,0,0,1
2,Male,African-American,24,0,0,1,4,1
3,Male,African-American,23,0,1,0,1,0
6,Male,Caucasian,41,0,0,0,14,1
8,Female,Caucasian,39,0,0,0,0,0
...,...,...,...,...,...,...,...,...
7208,Male,African-American,20,0,0,0,0,0
7209,Male,African-American,23,0,0,0,0,0
7210,Male,African-American,23,0,0,0,0,0
7212,Female,African-American,33,0,0,0,3,0


In [3]:
# Encode categorical variables
categorical = ['sex', 'race']
for feature in categorical:
    le = LabelEncoder()
    df_raw[feature] = le.fit_transform(df_raw[feature])
a = df_raw.drop(['sex', 'race', 'two_year_recid'], axis=1)
s = pd.DataFrame({'race-sex': df_raw.race * 2 + df_raw.sex})
y = pd.DataFrame({'two_year_recid': df_raw['two_year_recid']})

In [4]:
# Feature scaling
scaler = StandardScaler()
a = pd.DataFrame(scaler.fit_transform(a), columns=a.columns)
pca = PCA()
a = pd.DataFrame(pca.fit_transform(a), columns=a.columns)
a.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count
0,-0.596519,0.429056,0.061913,0.000674,0.308648
1,0.962933,1.163002,-0.443991,0.970232,-0.761187
2,0.929383,1.031140,-0.217031,-1.391831,0.936699
3,0.548070,-1.669156,-0.304386,-0.308827,-1.258837
4,-0.686701,0.121587,-0.060447,0.156054,0.494724


In [5]:
# Split data into separate training and test set
a_train, a_test, s_train, s_test, y_train, y_test = \
    train_test_split(a, s, y, test_size = 0.25, random_state = 0)

In [6]:
fairdata_ortho = FairData(s_train, a_train, y_train, preprocess_method='o')
fairdata_mdm = FairData(s_train, a_train, y_train, preprocess_method='m')

In [7]:
fairdata_mdm_eval = fairdata_mdm.evaluate(
    a_test, s_test, y_test, metrics=['cf', 'cfbm', 'acc'], p_range=0.05, b=50
)
pd.DataFrame(fairdata_mdm_eval, index=['CF', 'CFBM', 'ACC'], columns=['ML', 'FTU', 'FL', 'AA', 'FLAP-1', 'FLAP-2'])

,ML,FTU,FL,AA,FLAP-1,FLAP-2
CF,0.227366,0.140649,0.005434,0.005982,0.002579,0.002674
CFBM,0.601561,0.581854,0.497138,0.497669,0.407372,0.403314
ACC,0.574430,0.572642,0.559839,0.560940,0.560712,0.560662


In [8]:
fairdata_ortho_eval = fairdata_ortho.evaluate(
    a_test, s_test, y_test, metrics=['cf', 'cfbm', 'acc'], methods=['FLAP-1', 'FLAP-2'], p_range=0.05, b=50
)
pd.DataFrame(fairdata_ortho_eval, index=['CF', 'CFBM', 'ACC'], columns=['FLAP-1', 'FLAP-2'])

,FLAP-1,FLAP-2
CF,0.005846,0.005434
CFBM,0.483640,0.483532
ACC,0.560502,0.559839


In [16]:
M = 50
res = np.empty((M, 3, 8))
for i in range(M):
    a_train, a_test, s_train, s_test, y_train, y_test = train_test_split(a, s, y, test_size = 0.25, random_state = i)
    fairdata_ortho = FairData(s_train, a_train, y_train, preprocess_method='o')
    fairdata_mdm = FairData(s_train, a_train, y_train, preprocess_method='m')
    fairdata_mdm_eval = fairdata_mdm.evaluate(
        a_test, s_test, y_test, metrics=['cf', 'cfbm', 'acc'], p_range=0.05, b=50
    )
    fairdata_ortho_eval = fairdata_ortho.evaluate(
        a_test, s_test, y_test, metrics=['cf', 'cfbm', 'acc'], methods=['FLAP-1', 'FLAP-2'], p_range=0.05, b=50
    )
    res[i] = np.concatenate([np.array(fairdata_mdm_eval), np.array(fairdata_ortho_eval)], axis=1)

In [20]:
res.mean(0).round(4)

array([[0.232 , 0.1422, 0.005 , 0.0054, 0.0032, 0.0032, 0.0053, 0.005 ],
       [0.6284, 0.5845, 0.483 , 0.4868, 0.4712, 0.469 , 0.4863, 0.4836],
       [0.5726, 0.5708, 0.559 , 0.5602, 0.5602, 0.5602, 0.5596, 0.559 ]])

In [22]:
res.std(0).round(4)

array([[0.0233, 0.006 , 0.0014, 0.0016, 0.0011, 0.0011, 0.0015, 0.0014],
       [0.0329, 0.0261, 0.0291, 0.0274, 0.0989, 0.0971, 0.0306, 0.0313],
       [0.0025, 0.0025, 0.0026, 0.0027, 0.0026, 0.0025, 0.0026, 0.0026]])

In [23]:
M = 100
res = np.empty((M, 3, 8))
for i in range(M):
    a_train, a_test, s_train, s_test, y_train, y_test = train_test_split(a, s, y, test_size = 0.25, random_state = i)
    fairdata_ortho = FairData(s_train, a_train, y_train, preprocess_method='o')
    fairdata_mdm = FairData(s_train, a_train, y_train, preprocess_method='m')
    fairdata_mdm_eval = fairdata_mdm.evaluate(
        a_test, s_test, y_test, metrics=['cf', 'cfbm', 'acc'], p_range=0.05, b=50
    )
    fairdata_ortho_eval = fairdata_ortho.evaluate(
        a_test, s_test, y_test, metrics=['cf', 'cfbm', 'acc'], methods=['FLAP-1', 'FLAP-2'], p_range=0.05, b=50
    )
    res[i] = np.concatenate([np.array(fairdata_mdm_eval), np.array(fairdata_ortho_eval)], axis=1)

In [24]:
res.mean(0)

array([[0.22655057, 0.14256565, 0.00509212, 0.00556072, 0.00317185,
        0.0031482 , 0.00544859, 0.00509212],
       [0.6229532 , 0.58129694, 0.48352854, 0.48691282, 0.4697829 ,
        0.4668709 , 0.48773714, 0.4857512 ],
       [0.57270307, 0.5709573 , 0.55904449, 0.56019371, 0.5603844 ,
        0.56036345, 0.55968109, 0.55904449]])

In [25]:
res.std(0)

array([[0.02179771, 0.00612776, 0.0013446 , 0.00149793, 0.00111465,
        0.00107077, 0.00147696, 0.0013446 ],
       [0.03366477, 0.02786099, 0.02951618, 0.02904584, 0.09277728,
        0.09199843, 0.03050259, 0.03094969],
       [0.00237429, 0.00239737, 0.00242831, 0.00251793, 0.00248421,
        0.00247472, 0.00246172, 0.00242831]])